In [1]:
import argparse
from typing import Any, List, Dict, Tuple, Callable

from GridWorld_environments import Grid_World
from RL_agents import ValueIterationAgent, QLearningAgent

from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from copy import deepcopy

### Value Function functionality

#### Perform Value Evaluation function

In [2]:
def perform_value_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], verbose=False):
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=GAMMA)
    
    while not vi_agent.value_converged:
        
        for state in gw_env.get_state_space():
            
            if state in gw_env.get_terminal_states():
                continue
                
            policy_act = policy[state]
            next_state = gw_env.get_new_state_on_action(old_state=state, action=policy_act)
            next_state_value = vi_agent.get_state_value(state=next_state)
            
            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + GAMMA * next_state_value))
        
    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    return vi_agent.get_value_function()

#### Train Value Iteration function (BIRL version)

In [3]:
def train_value_iteration(gw_env: Grid_World, 
                          policy: Dict[Any, Any] = None,
                          value_function: Dict[Any, float] = None,
                          verbose=False
                         ) -> Tuple[Dict[Any, Any], Dict[Any, float]]:
    
    vi_agent = ValueIterationAgent(states=gw_env.get_state_space(),
                                   terminal_states=gw_env.get_terminal_states(),
                                   reward_function=gw_env.get_reward_func(),
                                   actions=gw_env.get_action_space(),
                                   gamma=GAMMA)

    if policy and value_function:
        raise ValueError("Can't pass both policy and value_function arguments at the same time.")
    
    if policy:
        if verbose:
            print("Calculating value function of the given policy via value evaluation...")
        vi_agent.set_value_function(new_value_function=perform_value_evaluation(gw_env, policy, verbose))
        
    if value_function:
        if verbose:
            print("Using the given value function...")
        vi_agent.set_value_function(new_value_function=value_function)
    
    iters = 0
    while iters < VALUE_ITERATION_TRAINING_N and not vi_agent.value_converged:

        for state in gw_env.get_state_space():

            if state in gw_env.get_terminal_states():
                continue

            opt_act = vi_agent.get_optimal_action(action_state_pairs=gw_env.get_action_state_pairs(state=state))
            next_state = gw_env.get_new_state_on_action(old_state=state, action=opt_act)
            next_state_value = vi_agent.get_state_value(state=next_state)

            vi_agent.set_state_value(state=state, new_value=(gw_env.get_state_reward(state=next_state) + GAMMA * next_state_value))

        iters += 1

    if verbose:
        gw_env.display_value_function(value_func=vi_agent.get_value_function())

    vi_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=vi_agent.get_policy())

    return vi_agent.get_policy(), vi_agent.get_value_function()


### Q Function functionality

In [4]:
def perform_q_function_evaluation(gw_env: Grid_World, policy: Dict[Any, Any], rounding: int = 3):

    value_func = perform_value_evaluation(gw_env, policy)
    rewards = gw_env.get_reward_func()

    q_function = {}
    for state in gw_env.get_state_space():

        action_values = {}
        for action in gw_env.get_action_space():

            next_state = gw_env.get_new_state_on_action(old_state=state, action=action)
            action_values[action] = np.round(rewards[next_state] + GAMMA * value_func[next_state], rounding)
            
        q_function[state] = action_values

    return q_function
    

#### Train Q Learning function

In [5]:
def train_q_learning(gw_env: Grid_World, n_episodes=1000, verbose=False, policy="eps_greedy", eps=0.2, max_episode_len=100, gamma=0.95):
    ql_agent = QLearningAgent(states=gw_env.get_state_space(),
                              terminal_states=gw_env.get_terminal_states(),
                              reward_function=gw_env.get_reward_func(),
                              actions=gw_env.get_action_space(),
                              gamma=gamma)
    
    # init episodes
    episodes = []
    
    # Define state_space without terminal states for getting starting position
    state_space = deepcopy(gw_env.get_state_space()) # all states
    terminal_states = gw_env.get_terminal_states()
    for terminal_state in terminal_states:
        state_space.remove(terminal_state) # not non absorbing state_space
    
    for n in range(n_episodes):
        
        episode = []
        
        # random starting position
        start_idx = (np.random.choice(len(state_space)))
        start = state_space[start_idx]
        
        episode.append(start)
        
        i = 0
        terminal = False
        
        while ( ( i < max_episode_len ) and ( not terminal ) ):
            i += 1
            
            # Choose Action from S derived by given policy
            if policy == "eps_greedy":
                if np.random.uniform() < (1-eps):
                    # Choose greedy action -> highest Q-Value
                    chosen_action = ql_agent.get_greedy_action(episode[-1])
                else:
                    # Choose random action form action space
                    action_space = gw_env.get_action_space()
                    chosen_action = action_space[np.random.choice(len(action_space))]
            
            new_state = gw_env.get_new_state_on_action(episode[-1], chosen_action)
            
            # Reward is taken from Q_learning agent -> it knows the reward function from the environment
            ql_agent.update_Q_value(episode[-1], new_state, chosen_action)
            
            episode.append(new_state)
            
            if new_state in terminal_states:
                terminal = True
        
        episodes.append(episode)
    
    if verbose:
        gw_env.display_q_function(q_func=ql_agent.get_Q_function())

    ql_agent.construct_greedy_policy(gw_env.get_action_state_pairs)

    if verbose:
        gw_env.display_policy(policy=ql_agent.get_policy())

    return ql_agent.get_policy(), ql_agent.get_Q_function()


### BIRL functionality

#### Reward Space functionality

In [6]:
def get_random_reward(size: Tuple[int], r_max: float = 1.0, rounding: int = 3) -> np.ndarray:
        
    return np.random.uniform(low=-abs(r_max), high=abs(r_max), size=size).round(rounding)

def get_reward_neighbour(reward: np.ndarray, step_size: float, r_max: float = 1.0) -> np.ndarray:
    
    movement = np.random.randint(low=-1, high=2, size=reward.shape) # random array of -1, 0 and +1
    
    return (reward + step_size * movement).clip(min=-abs(r_max), max=abs(r_max)) # new neighbour reward

In [7]:
get_reward_neighbour(np.array([1, 1, 1, 1, 1, 1]), 0.2)

array([0.8, 1. , 1. , 0.8, 1. , 1. ])

#### Prior, Evidence and Posterior Distribution

In [8]:
def improper_prior(reward):
    return 1

def compute_log_posterior(gw_env:Grid_World,
                          observations: List[List[Tuple[Any]]],
                          reward: np.ndarray,
                          policy: Dict[Any, Any],
                          prior: Callable,
                          alpha: float,
                          gamma: float
                         ) -> float:
    
    gw_env.set_board(new_board=reward)
    q = perform_q_function_evaluation(gw_env=gw_env, policy=policy)
    
    log_p = 0
    
    for observation in observations:
        log_p += np.sum([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
        #### Should we use np.mean() here? our obs are not all of the same length
        # log_p += np.mean([alpha * q[s][a] - np.log(np.sum(np.exp(alpha * np.array(list(q[s].values()))))) for s, a in observation])
    
    log_p += np.log(prior(reward))
    
    return log_p

def posterior_dist_ratio(gw_env: Grid_World,
                         observations: List[List[Tuple[Any]]],
                         reward_next: np.ndarray,
                         policy_next: Dict[Any, Any],
                         reward_curr: np.ndarray,
                         policy_curr: Dict[Any, Any]
                        ) -> float:
    
    prior = improper_prior
    alpha = 10
    gamma = GAMMA
    
    log_post_next = compute_log_posterior(gw_env, observations, reward_next, policy_next, prior, alpha, gamma)
    log_post_curr = compute_log_posterior(gw_env, observations, reward_curr, policy_curr, prior, alpha, gamma)
    
    return np.exp(log_post_next - log_post_curr)


#### PolicyWalk sampling algorithm

Credits for helping to decypher the confusing formulation of the paper go to...

- https://github.com/uidilr/bayesian_irl

In [9]:
def policyWalk(observations: List[List[Tuple[Any]]],
               gw_env: Grid_World,
               step_size: float,
               n_steps: int,
               samples_burn_in: int = 500,
               samples_n_out: int = 5
              ) -> np.ndarray:
    
    # step 1: pick a random reward from the reward space (as np.array)
    reward_0 = get_random_reward(size=gw_env.get_board().shape)
    
    # step 2: get optimal policy given reward_0
    gw_env.set_board(new_board=reward_0)  # board is the np.array representation of the reward function
    policy_0, val_func_0 = train_value_iteration(gw_env=gw_env)
    
    posterior_samples = []
    
    # step 3: explore the reward space and sample from the posterior (GridWalk as a MCMC (Metropolis algorithm))
    for i in range(n_steps):
        # step 3a: pick reward_1 from reward_0 neighbours randomly (step_size distance)
        reward_1 = get_reward_neighbour(reward=reward_0, step_size=step_size)
        
        # step 3b: compute the Q function of reward_1 under policy_0
        gw_env.set_board(new_board=reward_1)
        q_func = perform_q_function_evaluation(gw_env=gw_env,
                                               policy=policy_0)
        
        # step 3c: check if ... 
        if is_policy_not_optimal(q_func=q_func, policy=policy_0,
                                 states=gw_env.get_state_space(), actions=gw_env.get_action_space()):
            # step 3c.i: get optimal policy given reward_1 AND policy_0 (we can use the value function of policy_0 directly)
            #### MOTIVATION: since reward_1 is a neighbour of reward_0, policy_1 will be close to policy_0. We use this fact to speed up the computation
            #### would be a repetition of step 3b: gw_env.set_board(new_board=reward_1)
            policy_1, val_func_1 = train_value_iteration(gw_env=gw_env,
                                                         value_function=val_func_0)
            
            # step 3c.ii: update reward_0 AND policy_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_1, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                policy_0 = deepcopy(policy_1)
                val_func_0 = deepcopy(val_func_1)
            
        else:
            # step 3c.iii: update reward_0 with prob. according to the posterior_dist
            if np.random.uniform() < np.minimum(1, posterior_dist_ratio(gw_env, observations, reward_1, policy_0, reward_0, policy_0)):
                reward_0 = deepcopy(reward_1)
                
        if (i > samples_burn_in) and (samples_n_out != 0) and ((i - samples_burn_in) % samples_n_out == 0):
            posterior_samples.append(reward_0)

    # step 4: return a sample from the posterior
    if samples_n_out != 0:
        return posterior_samples
    else:
        return reward_0

def is_policy_not_optimal(q_func, policy, states, actions):
    for state in states:
        for action in actions:
            if q_func[state][policy[state]] < q_func[state][action]:
                return True
    return False


### Step-by-step algorithm

In [10]:
GAMMA = 0.95

VALUE_ITERATION_TRAINING_N = 50

NUMBER_OF_TRAJECTORIES = 25
MAXIMUM_TRAJECTORY_LENGTH = 25

BIRL_N_OF_POSTERIOR_SAMPLES = 5
BIRL_POLICYWALK_N_STEPS = 1000
BIRL_POLICYWALK_STEP_SIZE = 0.05

GW_SIZE = (3, 3)
GW_SIZES = [(3, 3)]  # [(x, x) for x in np.arange(5,11, 5)]
GW_TRAPS = []
GW_GOALS = [(0, 0)]


In [11]:
environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)


In [12]:
vi_greedy_policy = train_value_iteration(gw_env=environment, verbose=True)[0]


Value function:
[[0.       1.       0.95    ]
 [1.       0.95     0.9025  ]
 [0.95     0.9025   0.857375]]
Policy:
[['x' '<' '<']
 ['^' '<' '<']
 ['^' '<' '<']]


In [13]:
print(f"Generating {NUMBER_OF_TRAJECTORIES} trajectories...")

trajectories = environment.generate_trajectories(policy=vi_greedy_policy,
                                                 n_traj=NUMBER_OF_TRAJECTORIES,
                                                 max_traj_length=MAXIMUM_TRAJECTORY_LENGTH,
                                                 return_state_action_pairs=True)


Generating 25 trajectories...


In [14]:
print(f"Generate {BIRL_N_OF_POSTERIOR_SAMPLES} samples from the posterior")

posterior_samples = []

for _ in tqdm(range(BIRL_N_OF_POSTERIOR_SAMPLES)):
    
    # restart the environment
    environment = Grid_World(size=GW_SIZE, traps=GW_TRAPS, goals=GW_GOALS, randomize_board=False)

    posterior_samples += policyWalk(observations=trajectories,
                                    gw_env=environment,
                                    step_size=BIRL_POLICYWALK_STEP_SIZE,
                                    n_steps=BIRL_POLICYWALK_N_STEPS)

Generate 5 samples from the posterior


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:21<00:00, 16.30s/it]


In [15]:
posterior_samples

[array([[ 0.25 , -0.6  , -0.6  ],
        [ 0.093, -1.   , -0.95 ],
        [-0.427, -0.808, -0.332]]),
 array([[ 0.3  , -0.65 , -0.55 ],
        [ 0.093, -1.   , -1.   ],
        [-0.577, -0.808, -0.382]]),
 array([[ 0.35 , -0.55 , -0.5  ],
        [ 0.143, -1.   , -0.95 ],
        [-0.677, -0.808, -0.432]]),
 array([[ 0.3  , -0.55 , -0.6  ],
        [ 0.093, -1.   , -0.9  ],
        [-0.627, -0.758, -0.382]]),
 array([[ 0.2  , -0.6  , -0.55 ],
        [ 0.143, -1.   , -0.9  ],
        [-0.677, -0.758, -0.382]]),
 array([[ 0.2  , -0.6  , -0.45 ],
        [ 0.093, -1.   , -0.85 ],
        [-0.677, -0.658, -0.382]]),
 array([[ 0.1  , -0.65 , -0.5  ],
        [ 0.143, -1.   , -0.85 ],
        [-0.727, -0.508, -0.382]]),
 array([[ 0.15 , -0.65 , -0.5  ],
        [ 0.143, -0.95 , -0.8  ],
        [-0.727, -0.408, -0.382]]),
 array([[ 0.1  , -0.65 , -0.45 ],
        [ 0.093, -0.95 , -0.85 ],
        [-0.777, -0.508, -0.382]]),
 array([[ 0.1  , -0.75 , -0.55 ],
        [ 0.093, -0.9  , -0.9 

In [16]:
np.mean(posterior_samples, axis=0)

array([[ 0.32293535, -0.78117576, -0.53612929],
       [ 0.12366465, -0.81532121, -0.73123232],
       [-0.54328687, -0.50098586, -0.55174949]])